In [1]:
import graphlab as gl
import datetime as dt

[WARNING] graphlab.product_key: Unable to write current GraphLab Create license to /home/nawel/.graphlab/config. Ensure that this user account                         has write permission to /home/nawel/.graphlab/config to save the license for offline use.
[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1482852630.log


This non-commercial license of GraphLab Create for academic use is assigned to nawel.medjkoune@u-psud.fr and will expire on December 27, 2017.


[WARNING] graphlab.deploy._session: Unable to create session in specified location: '/home/nawel/.graphlab/artifacts'. Using: '/var/tmp/graphlab-nawel/4819/tmp_session_7930b898-a3b3-4aba-83c5-bc547064c2da'


In [2]:
data=gl.SFrame.read_csv('../../../train_2011_2012_2013.csv', delimiter=';')


Finished parsing file /media/nawel/E4DC5C32DC5C016A/AIC/Big data machine learning/Project/train_2011_2012_2013.csv

Parsing completed. Parsed 100 lines in 3.22818 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str,int,str,str,int,int,int,str,str,str,str,str,str,str,str,str,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 163398 lines. Lines per second: 44812.9

Read 642804 lines. Lines per second: 67882

...

Read 9290581 lines. Lines per second: 88786

Read 9780525 lines. Lines per second: 89200

Read 10431918 lines. Lines per second: 89871.5

Finished parsing file /media/nawel/E4DC5C32DC5C016A/AIC/Big data machine learning/Project/train_2011_2012_2013.csv

Parsing completed. Parsed 10878470 lines in 119.972 secs.

In [ ]:
#2011-04-24 01:30:00.000
data['DATE'] = data['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))

In [ ]:
data_ts = gl.TimeSeries(data, index="DATE")


In [ ]:
start = dt.datetime(2011,1,1)
end  = dt.datetime(2012,1,1)
ts_2011 = data_ts.slice(start, end)

In [3]:
model = gl.boosted_trees_regression.create(data, target='CSPL_RECEIVED_CALLS', features=['DATE', 'DAY_OFF', 'WEEK_END', 'DAY_WE_DS', 'ASS_ASSIGNMENT'])


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: Detected extremely low variance for feature(s) 'DAY_OFF' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees regression:

--------------------------------------------------------

Number of examples          : 10334186

...

| 7         | 28.731266    | 444.502502         | 697.030273           | 4.374564      | 4.425839        |

| 8         | 33.227279    | 426.866364         | 695.446838           | 4.370176      | 4.424080        |

| 9         | 36.922302    | 419.031952         | 694.014526           | 4.367849      | 4.424621        |

| 10        | 41.114151    | 419.033752         | 690.135132           | 4.365370      | 4.424082        |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

In [4]:
test=gl.SFrame.read_csv('../../../submission.txt', delimiter='\t')

Finished parsing file /media/nawel/E4DC5C32DC5C016A/AIC/Big data machine learning/Project/submission.txt

Parsing completed. Parsed 100 lines in 0.278843 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /media/nawel/E4DC5C32DC5C016A/AIC/Big data machine learning/Project/submission.txt

Parsing completed. Parsed 82909 lines in 0.095204 secs.

In [5]:
test=test['DATE' , 'ASS_ASSIGNMENT']
#test['DATE'] = test['DATE'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000'))
test

DATE,ASS_ASSIGNMENT
2012-12-28 00:00:00.000,CMS
2012-12-28 00:00:00.000,Crises
2012-12-28 00:00:00.000,Domicile
2012-12-28 00:00:00.000,Gestion
2012-12-28 00:00:00.000,Gestion - AccueilTelephonique ...
2012-12-28 00:00:00.000,Gestion Assurances
2012-12-28 00:00:00.000,Gestion RelationClienteles ...
2012-12-28 00:00:00.000,Gestion Renault
2012-12-28 00:00:00.000,Japon
2012-12-28 00:00:00.000,Médical


In [ ]:
model = gl.boosted_trees_regression.create(data, target='CSPL_RECEIVED_CALLS', features=['DAY_OFF', 'WEEK_END', 'DAY_WE_DS', 'ASS_ASSIGNMENT'], max_iterations=2, max_depth =  3)

In [ ]:
labels = data['CSPL_RECEIVED_CALLS'].unique()

In [6]:
predictions=model.predict(test)
#predictions=predictions.apply(lambda x: int(x))

In [ ]:
import random
listOfNumbers=[]
for x in range (0,82908):
    listOfNumbers.append(random.randint(0, 1000))


In [9]:
def make_submission(prediction, filename='submission.txt'):
    with open(filename, 'w') as f:
        f.write('DATE\tASS_ASSIGNMENT\tprediction\n')
        submission_strings = test['DATE'] + '\t' + test['ASS_ASSIGNMENT'] + '\t'+ prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n')

make_submission(predictions, 'submission.txt')


